In [ ]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import seaborn as sns
from collections import Counter
from copy import deepcopy as dc
from itertools import chain
from sympy import Derivative, symbols

$f(x) = (x_1 + x_2)^2 + 3$ <br>
$subject \,to $ <br>
    $\quad g1(x) = x_1 + x_2 >= 5$ <br>
    $\quad g2(x) = 2x_1 + 3x_2 <= 12$ <br>
    $\quad x_1>=0, x_2>=0$

$It \; can \; be \; defined \; as$ <br>
$f(x) = (x_1 + x_2)^2 + 3$ <br><br>
$subject \,to $ <br>
    $\quad h_1(x) = x_1 + x_2 - s_1= 5$ <br>
    $\quad h_2(x) = 2x_1 + 3x_2 + s_2= 12$ <br>
    $\quad x_1>=0, x_2>=0, s_1>=0, s_2>=0$

step 1

In [ ]:
# define objective and constrain function
def f(x1,x2):
    return x1*x1 + x2*x2 + 2*x1*x2 + 3

def h1(x1,x2,s1):
    return np.array(x1 + x2 - s1 - 5)

def h2(x1,x2,s2):
    return np.array(2*x1 + 3*x2 + s2 - 12)

def h3(x1,x2,s1,s2):
    if (x1<=0) | (x2<=0) | (s1<=0) | (s2<=0): return False
    else: return True

In [ ]:
# initial start point
x1 = 3; x2 = 2 ; s1 = 0; s2=0
print(f'x1={x1}, x2={x2}, s1={s1}, s2={s2}')

x1=3, x2=2, s1=0, s2=0


In [ ]:
# define termination condition
epsilon = 0.1

step 2

In [ ]:
var_ = ['x1','x2','s1','s2']

In [ ]:
# basic / nonbasic variable
x=[x1,x2,s1,s2]
print( np.argsort(x)[::-1] )

# 0,1  / 2,3
x_j = x[:2]; x_c = [i for i in [x1,x2,s1,s2] if i not in x_j ]
print(f'x_j={x_j}, x_c={x_c}')

[0 1 3 2]
x_j=[3, 2], x_c=[0, 0]


In [ ]:
# define J, C

def deriv_h1(x, p):
    if x==var_[0]:
        x = symbols(var_[0])
        hx = x+x2+s1-5
    elif x==var_[1]:
        x = symbols(var_[1])
        hx = x1+x+s1-5
    else:
        x = symbols(var_[1])
        hx = x1+x2+x-5
    fprime = Derivative(hx, x).doit() # x에 대해서 미분
    return np.array(fprime.subs({x: p})).tolist()

def deriv_h2(x, p):
    if x==var_[0]:
        x = symbols(var_[0])
        hx = 2*x+3*x2+s2-12
    elif x==var_[1]:
        x = symbols(var_[1])
        hx = 2*x1+3*x+s2-12
    else:
        x = symbols(var_[1])
        hx = 2*x1+3*x2+x-12
    fprime = Derivative(hx, x).doit() # x에 대해서 미분
    return np.array(fprime.subs({x: p})).tolist()

In [ ]:
J = np.array( [[deriv_h1(var_[0], x1),deriv_h1(var_[1], x2)], [deriv_h2(var_[0], x1),deriv_h2(var_[1], x2)]], dtype=np.float32 )
C = np.array( [[deriv_h1(var_[2], s1),deriv_h1(var_[3], s2)], [deriv_h2(var_[2], s1),deriv_h2(var_[3], s2)]], dtype=np.float32 )
print(J)
print(C)

[[1. 1.]
 [2. 3.]]
[[1. 1.]
 [1. 1.]]


In [ ]:
def deriv_f(x, p):
    if x==var_[0]:
        x = symbols(var_[0])
        fx = x*x + x2*x2 + 2*x*x2 + 3
    elif x==var_[1]:
        x = symbols(var_[1])
        fx = x1*x1 + x*x + 2*x1*x + 3
    fprime = Derivative(fx, x).doit() #x에 대해서 미분
    return np.array(fprime.subs({x: p})).tolist()

In [ ]:
df_j = np.array( [deriv_f(var_[0], x1),deriv_f(var_[1], x2)] )
df_c = [0, 0]

# reduced gradient
r = df_c - df_j@np.linalg.inv(J)@np.array(C)

step 3

In [ ]:
def l2_norm(x):
    return np.sqrt(float(np.sum(x**2)))

if l2_norm(r) > epsilon:
    # setting direction
    d_c = -r
    d_j = -np.linalg.inv(J)@C@d_c
    d = np.array(d_j.tolist() + d_c.tolist())

d

array([-40.0000000000000, 20.0000000000000, 10.0000000000000,
       10.0000000000000], dtype=object)

step 4

In [ ]:
alpha = 1; gamma = 0.8

In [ ]:
# step 4-a
keep_go = True

while keep_go:

    print("\nStep 4-a start")
    v = [3,2,0,0] + alpha*d

    print(v)

    if (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2],v[3]):
        print('go to 4-d')
        break
    else:
        print('go to 4-b')


    # step 4-b
    print("\nStep 4-b start")
    v_j = v[:2]
    v_j -= np.array( [[deriv_h1(var_[0], v[0]),deriv_h1(var_[1], v[1])], [deriv_h2(var_[0], v[0]),deriv_h2(var_[1], v[1])]], dtype=np.float32 ) @ np.array([h1(v[0],v[1],v[2]), h2(v[0],v[1],v[3])], dtype=np.float32)
    v_c = v[2:]
    v_prev = dc(v)
    v[:2] = v_j
    print(v)

    # step 4-c
    print("\nStep 4-c start")
    if abs(l2_norm(v-v_prev)) > epsilon:
        print('go to 4-b')
    elif (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2], v[3]):
        print('go to 4-d')
        print(keep_go=False)
        print("???????")
    else:
        alpha *= gamma
        print('go to 4-a')
        print(f'alpha = {alpha}')




Step 4-a start
[-37.0000000000000 22.0000000000000 10.0000000000000 10.0000000000000]
go to 4-b

Step 4-b start
[3.00000000000000 112.000000000000 10.0000000000000 10.0000000000000]

Step 4-c start
go to 4-a
alpha = 0.8

Step 4-a start
[-29.0000000000000 18.0000000000000 8.00000000000000 8.00000000000000]
go to 4-b

Step 4-b start
[3.00000000000000 90.0000000000000 8.00000000000000 8.00000000000000]

Step 4-c start
go to 4-a
alpha = 0.6400000000000001

Step 4-a start
[-22.6000000000000 14.8000000000000 6.40000000000000 6.40000000000000]
go to 4-b

Step 4-b start
[3.00000038146972 72.4000022888184 6.40000000000000 6.40000000000000]

Step 4-c start
go to 4-a
alpha = 0.5120000000000001

Step 4-a start
[-17.4800000000000 12.2400000000000 5.12000000000000 5.12000000000000]
go to 4-b

Step 4-b start
[2.99999954223632 58.3199980163574 5.12000000000000 5.12000000000000]

Step 4-c start
go to 4-a
alpha = 0.40960000000000013

Step 4-a start
[-13.3840000000000 10.1920000000000 4.09600000000000 4

In [ ]:
# step 4-d
print(f'previous: {f(3,2)}, now: {f(v[0],v[1])}')
if f(3,2) <= f(v[0],v[1]):
    print("not optimized")
    # go to step 4-a
    alpha *= gamma
else:
    print('optimized! go to step2')
    print(v)

previous: 28, now: 27.3991908442860
optimized! go to step2
[2.87910741803854 2.06044629098073 0.0302231454903658 0.0302231454903658]


repeat until convergence

In [ ]:

# step 2
# basic / nonbasic variable
x=dc(v)
print(x)
print( np.argsort(v)[::-1] )

# 0,1  / 2,3
x_j = x[:2]; x_c = [i for i in x if i not in x_j ]
print(f'x_j={x_j}, x_c={x_c}')
J = np.array( [[deriv_h1(var_[0], x[0]),deriv_h1(var_[1], x[1])], [deriv_h2(var_[0], x[0]),deriv_h2(var_[1], x[1])]], dtype=np.float32 )
C = np.array( [[deriv_h1(var_[2], x[2]),deriv_h1(var_[3], x[3])], [deriv_h2(var_[2], x[2]),deriv_h2(var_[3], x[3])]], dtype=np.float32 )


df_j = np.array( [deriv_f(var_[0], x[0]),deriv_f(var_[1], x[1])] )
df_c = [0, 0]

# reduced gradient
r = df_c - df_j@np.linalg.inv(J)@np.array(C)

# step 3
if l2_norm(r) > epsilon:
    # setting direction
    d_c = -r
    d_j = -np.linalg.inv(J)@C@d_c
    d = np.array(d_j.tolist() + d_c.tolist())
    print(f'direction: {d}')
else:
    print('convergence!')



[2.87910741803854 2.06044629098073 0.0302231454903658 0.0302231454903658]
[0 1 3 2]
x_j=[2.87910741803854 2.06044629098073], x_c=[0.0302231454903658, 0.0302231454903658]
direction: [-37.5821483607707 18.7910741803854 9.39553709019268 9.39553709019268]


In [ ]:
# step 4
alpha = 1; gamma = 0.8
# step 4-a
keep_go = True

while keep_go:

    print("\nStep 4-a start")
    v = x + alpha*d

    print(v)

    if (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2],v[3]):
        print('go to 4-d')
        break
    else:
        print('go to 4-b')


    # step 4-b
    print("\nStep 4-b start")
    v_j = v[:2]
    v_j -= np.array( [[deriv_h1(var_[0], v[0]),deriv_h1(var_[1], v[1])], [deriv_h2(var_[0], v[0]),deriv_h2(var_[1], v[1])]], dtype=np.float32 ) @ np.array([h1(v[0],v[1],v[2]), h2(v[0],v[1],v[3])], dtype=np.float32)
    v_c = v[2:]
    v_prev = dc(v)
    v[:2] = v_j
    print(v)

    # step 4-c
    print("\nStep 4-c start")
    if abs(l2_norm(v-v_prev)) > epsilon:
        print('go to 4-b')
    elif (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2], v[3]):
        print('go to 4-d')
        print(keep_go=False)
    else:
        alpha *= gamma
        print('go to 4-a')
        print(f'alpha = {alpha}')


Step 4-a start
[-34.7030409427322 20.8515204713661 9.42576023568305 9.42576023568305]
go to 4-b

Step 4-b start
[3.00000013392796 105.683360986503 9.42576023568305 9.42576023568305]

Step 4-c start
go to 4-a
alpha = 0.8

Step 4-a start
[-27.1866112705780 17.0933056352890 7.54665281764451 7.54665281764451]
go to 4-b

Step 4-b start
[2.99999990495906 85.0131817339218 7.54665281764451 7.54665281764451]

Step 4-c start
go to 4-a
alpha = 0.6400000000000001

Step 4-a start
[-21.1734675328547 14.0867337664274 6.04336688321368 6.04336688321368]
go to 4-b

Step 4-b start
[3.00000010325366 68.4770345171598 6.04336688321368 6.04336688321368]

Step 4-c start
go to 4-a
alpha = 0.5120000000000001

Step 4-a start
[-16.3629525426761 11.6814762713380 4.84073813566902 4.84073813566902]
go to 4-b

Step 4-b start
[3.00000064335907 55.2481190325685 4.84073813566902 4.84073813566902]

Step 4-c start
go to 4-a
alpha = 0.40960000000000013

Step 4-a start
[-12.5145405505332 9.75727027526658 3.87863513763329 3

In [ ]:
# step 4-d
print(f'previous: {f(3,2)}, now: {f(v[0],v[1])}')
if f(3,2) <= f(v[0],v[1]):
    print("\nnot optimized")
    # go to step 4-a
    alpha *= gamma
else:
    print('\noptimized! go to step2')
    print(v)

previous: 28, now: 27.3389847176569

optimized! go to step2
[2.86691131360912 2.06654434319544 0.0332721715977192 0.0332721715977192]


In [ ]:
real_go = True
while real_go:
    # step 2
    # basic / nonbasic variable
    x=dc(v)
    print(x)
    print( np.argsort(v)[::-1] )

    # 0,1  / 2,3
    x_j = x[:2]; x_c = [i for i in x if i not in x_j ]
    print(f'x_j={x_j}, x_c={x_c}')
    J = np.array( [[deriv_h1(var_[0], x[0]),deriv_h1(var_[1], x[1])], [deriv_h2(var_[0], x[0]),deriv_h2(var_[1], x[1])]], dtype=np.float32 )
    C = np.array( [[deriv_h1(var_[2], x[2]),deriv_h1(var_[3], x[3])], [deriv_h2(var_[2], x[2]),deriv_h2(var_[3], x[3])]], dtype=np.float32 )


    df_j = np.array( [deriv_f(var_[0], x[0]),deriv_f(var_[1], x[1])] )
    df_c = [0, 0]

    # reduced gradient
    r = df_c - df_j@np.linalg.inv(J)@np.array(C)

    # step 3
    if l2_norm(r) > epsilon:
        # setting direction
        d_c = -r
        d_j = -np.linalg.inv(J)@C@d_c
        d = np.array(d_j.tolist() + d_c.tolist())
        print(f'direction: {d}')
    else:
        print('convergence!')
        real_go = False
        break

    # step 4
    alpha = 1; gamma = 0.8
    # step 4-a
    keep_go = True

    while keep_go:

        print("\nStep 4-a start")
        v = x + alpha*d

        print(v)

        if (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2],v[3]):
            print('go to 4-d')
            break
        else:
            print('go to 4-b')


        # step 4-b
        print("\nStep 4-b start")
        v_j = v[:2]
        v_j -= np.array( [[deriv_h1(var_[0], v[0]),deriv_h1(var_[1], v[1])], [deriv_h2(var_[0], v[0]),deriv_h2(var_[1], v[1])]], dtype=np.float32 ) @ np.array([h1(v[0],v[1],v[2]), h2(v[0],v[1],v[3])], dtype=np.float32)
        v_c = v[2:]
        v_prev = dc(v)
        v[:2] = v_j
        print(v)

        # step 4-c
        print("\nStep 4-c start")
        if abs(l2_norm(v-v_prev)) > epsilon:
            print('go to 4-b')
        elif (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2], v[3]):
            print('go to 4-d')
            print(keep_go=False)
        else:
            alpha *= gamma
            print('go to 4-a')
            print(f'alpha = {alpha}')

    step4_d = True
    while step4_d:
        # step 4-d
        print(f'previous: {f(3,2)}, now: {f(v[0],v[1])}')
        if f(3,2) <= f(v[0],v[1]):
            print("\nnot optimized")
            # go to step 4-a
            alpha *= gamma
        else:
            print('\noptimized! go to step2')
            print(v)
            step4_d = False
            keep_go = False

        while keep_go:

            print("\nStep 4-a start")
            v = x + alpha*d

            print(v)

            if (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2],v[3]):
                print('go to 4-d')
                break
            else:
                print('go to 4-b')


            # step 4-b
            print("\nStep 4-b start")
            v_j = v[:2]
            v_j -= np.array( [[deriv_h1(var_[0], v[0]),deriv_h1(var_[1], v[1])], [deriv_h2(var_[0], v[0]),deriv_h2(var_[1], v[1])]], dtype=np.float32 ) @ np.array([h1(v[0],v[1],v[2]), h2(v[0],v[1],v[3])], dtype=np.float32)
            v_c = v[2:]
            v_prev = dc(v)
            v[:2] = v_j
            print(v)

            # step 4-c
            print("\nStep 4-c start")
            if abs(l2_norm(v-v_prev)) > epsilon:
                print('go to 4-b')
            elif (abs( h1(v[0],v[1],v[2]) ) <= epsilon) & (abs( h2(v[0],v[1],v[3]) ) <= epsilon) & h3(v[0],v[1],v[2], v[3]):
                print('go to 4-d')
                print(keep_go=False)
            else:
                alpha *= gamma
                print('go to 4-a')
                print(f'alpha = {alpha}')

In [ ]:
x=dc(v)

In [ ]:
print(x)
print( f(x[0],x[1]) )

print( h1(x[0],x[1],x[2]) )
print( h2(x[0],x[1],x[3]) )
